In [1]:
## Load relevant R packages.

inLibraries = list('repr','spdep','raster','rgdal','rgeos')
for (rpack in inLibraries) {
  if (is.element(rpack,installed.packages()[,1])){
      #Load the library into R
      suppressMessages(library(rpack,character.only = TRUE))
    }
    else {
        print(paste("Warning:  ",rpack," is not an installed package"))
    }
}

##  Resize plot area.

options(repr.plot.width=6, repr.plot.height=6)

In [2]:
lc1700name <- 'data/lc/historic_landcover_hd_1700.asc' 
lc1700 <- raster(lc1700name,values=TRUE)

lc1750name <- "data/lc/historic_landcover_hd_1750.asc"
lc1750 <- raster(lc1750name,values=TRUE)

lc1800name <- "data/lc/historic_landcover_hd_1800.asc"
lc1800 <- raster(lc1800name,values=TRUE)

lc1850name <- "data/lc/historic_landcover_hd_1850.asc"
lc1850 <- raster(lc1850name,values=TRUE)

lc1900name <- "data/lc/historic_landcover_hd_1900.asc"
lc1900 <- raster(lc1900name,values=TRUE)

lc1950name <- "data/lc/historic_landcover_hd_1950.asc"
lc1950  <- raster(lc1950name,values=TRUE)

lc1970name <- "data/lc/historic_landcover_hd_1970.asc"
lc1970 <- raster(lc1970name,values=TRUE)

lc1990name <- "data/lc/historic_landcover_hd_1990.asc"
lc1990 <- raster(lc1990name,values=TRUE)


meteorites <- readOGR("data/results", "meteorites_sf")

head(meteorites)

OGR data source with driver: ESRI Shapefile 
Source: "data/results", layer: "meteorites_sf"
with 9796 features
It has 14 fields


X,name,id,nametyp,recclss,mss_n_g,fll_r_f,year,GeoLctn,grop_nm,Chndr_A,type,L3,L4
5625,Aachen,1,Valid,L5,21,Fell,1880,"(50.775, 6.08333)",L,Chondrite,Stony,Ordinary,L
568,Aarhus,2,Valid,H6,720,Fell,1951,"(56.18333, 10.23333)",H,Chondrite,Stony,Ordinary,H
916,Abajo,4,Valid,H5,331,Found,1982,"(26.8, -105.41667)",H,Chondrite,Stony,Ordinary,H
918,Abbott,5,Valid,H3-6,21100,Found,1951,"(36.3, -104.28333)",H,Chondrite,Stony,Ordinary,H
399,Abee,6,Valid,EH4,107000,Fell,1952,"(54.21667, -113.0)",EH,Chondrite,Stony,Enstatite,EH-EL
6004,Abernathy,7,Valid,L6,2914,Found,1941,"(33.85, -101.8)",L,Chondrite,Stony,Ordinary,L


In [3]:
landcover_table <- data.frame(coordinates(meteorites),
                              meteorites$name, 
                              meteorites$id,
                              meteorites$year,
                              extract(lc1700, meteorites),
                              extract(lc1750, meteorites),
                              extract(lc1800, meteorites),
                              extract(lc1850, meteorites),
                              extract(lc1900, meteorites),
                              extract(lc1950, meteorites),
                              extract(lc1970, meteorites),
                              extract(lc1990, meteorites))

names(landcover_table) <- c("x", "y", "name", "id", "year","lc1700","lc1750","lc1800","lc1850","lc1900","lc1950","lc1970","lc1990")

head(landcover_table)

x,y,name,id,year,lc1700,lc1750,lc1800,lc1850,lc1900,lc1950,lc1970,lc1990
6.08333,50.77500,Aachen,1,1880,11,11,11,11,11,11,11,11
10.23333,56.18333,Aarhus,2,1951,1,1,1,1,1,1,1,1
-105.41667,26.80000,Abajo,4,1982,15,15,15,15,15,15,2,2
-104.28333,36.30000,Abbott,5,1951,13,13,13,13,13,13,13,13
-113.00000,54.21667,Abee,6,1952,9,9,9,9,9,2,1,1
-101.80000,33.85000,Abernathy,7,1941,13,13,13,13,1,1,1,1


In [4]:
meteorites_pre1700  <- meteorites[meteorites$year < 1700,]
meteorites_post1700  <- meteorites[meteorites$year > 1700,]

dim(meteorites)
dim(meteorites_post1700)
dim(meteorites_pre1700)

meteorites_pre1700

[1] 9796   14

[1] 9774   14

[1] 22 14

                coordinates    X             name    id nametyp      recclss
847  (-60.58333, -27.46667) 4820  Campo del Cielo  5247   Valid Iron, IAB-MG
888            (16.2, 39.8) 9660    Castrovillari  5295   Valid   Stone-uncl
3275   (12.73333, 50.18333) 5070          Elbogen  7823   Valid    Iron, IID
3302       (7.35, 47.86667) 8863        Ensisheim 10039   Valid          LL6
3435   (10.33333, 55.33333) 9665     F\303\274nen 10838   Valid   Stone-uncl
3969      (-1.51667, 51.65) 9668          Hatford 11855   Valid   Stone-uncl
4166               (75, 31) 5390        Jalandhar 12069   Valid         Iron
4787   (136.9333, 35.07833) 5848         Minamino 16692   Valid            L
4832     (-105.4333, 27.05) 5278           Morito 16745   Valid  Iron, IIIAB
4865    (6.86667, 44.08333) 9671      Mount Vaisi 16805   Valid   Stone-uncl
4958   (12.51667, 42.51667) 9673            Narni 16914   Valid   Stone-uncl
5020       (130.75, 33.725) 5873           Nogata 16988   Valid           L6

In [9]:
# Only 22 meteorites (0.2%) are pre 1700 and could represent significantly different landcover from the 1700 raster.  
# Not a major issue.